## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
%matplotlib inline

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Adrar,MR,20.5022,-10.0711,92.66,26,61,9.93,broken clouds
1,1,Yarada,IN,17.6500,83.2667,84.52,84,75,5.75,mist
2,2,Punta Arenas,CL,-53.1500,-70.9167,39.31,81,0,27.63,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,37.06,75,75,3.44,broken clouds
4,4,Acandi,CO,8.5063,-77.2778,77.31,89,17,3.00,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

preferred_cities_df.head(20)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Yarada,IN,17.6500,83.2667,84.52,84,75,5.75,mist
4,4,Acandi,CO,8.5063,-77.2778,77.31,89,17,3.00,few clouds
5,5,Yellowknife,CA,62.4560,-114.3525,73.92,49,75,6.91,broken clouds
8,8,Atuona,PF,-9.8000,-139.0333,76.73,78,73,17.87,light rain
12,12,Galesong,ID,-5.3166,119.3661,84.22,72,71,5.50,broken clouds
22,22,Surt,LY,31.2089,16.5887,79.45,69,0,7.65,clear sky
24,24,Yeppoon,AU,-23.1333,150.7333,73.18,61,0,7.47,clear sky
27,27,Thompson,CA,55.7435,-97.8558,72.75,45,8,5.50,clear sky
28,28,Saint-Georges,GD,12.0564,-61.7485,79.65,80,7,14.74,clear sky
29,29,Vaini,TO,-21.2000,-175.2000,78.96,83,40,13.80,scattered clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                313
City                   313
Country                313
Lat                    313
Lng                    313
Max Temp               313
Humidity               313
Cloudiness             313
Wind Speed             313
Current Description    313
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                313
City                   313
Country                313
Lat                    313
Lng                    313
Max Temp               313
Humidity               313
Cloudiness             313
Wind Speed             313
Current Description    313
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yarada,IN,84.52,mist,17.6500,83.2667,
4,Acandi,CO,77.31,few clouds,8.5063,-77.2778,
5,Yellowknife,CA,73.92,broken clouds,62.4560,-114.3525,
8,Atuona,PF,76.73,light rain,-9.8000,-139.0333,
12,Galesong,ID,84.22,broken clouds,-5.3166,119.3661,
22,Surt,LY,79.45,clear sky,31.2089,16.5887,
24,Yeppoon,AU,73.18,clear sky,-23.1333,150.7333,
27,Thompson,CA,72.75,clear sky,55.7435,-97.8558,
28,Saint-Georges,GD,79.65,clear sky,12.0564,-61.7485,
29,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in clean_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (Exception):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.drop(hotel_df.loc[hotel_df['Hotel Name'] == 1].index)

clean_hotel_df.head(33)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yarada,IN,84.52,mist,17.6500,83.2667,Yarada Jungle Beach Resorts
4,Acandi,CO,77.31,few clouds,8.5063,-77.2778,Hotel Patio Bonito
5,Yellowknife,CA,73.92,broken clouds,62.4560,-114.3525,The Explorer Hotel
8,Atuona,PF,76.73,light rain,-9.8000,-139.0333,Villa Enata
12,Galesong,ID,84.22,broken clouds,-5.3166,119.3661,Sapriadi Appy
22,Surt,LY,79.45,clear sky,31.2089,16.5887,الهادي ابوشعاله سرت
24,Yeppoon,AU,73.18,clear sky,-23.1333,150.7333,Villa Mar Colina
27,Thompson,CA,72.75,clear sky,55.7435,-97.8558,Burntwood Hotel
28,Saint-Georges,GD,79.65,clear sky,12.0564,-61.7485,Radisson Grenada Beach Resort
29,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort


In [43]:
# 8a. Create the output File (CSV)

output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))